<a href="https://colab.research.google.com/github/RyuMinSu/project/blob/master/kci__API_%ED%99%9C%EC%9A%A9_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import urllib
from urllib.parse import quote_plus

from bs4 import BeautifulSoup as bs
import time

import pandas as pd

from tqdm import tqdm
from tqdm import trange

############################# api 사용 매뉴얼 ################################
print('#'*20, 'API 사용 메뉴얼', '#'*20)

#키
key = 
years = [2019, 2020, 2021]
#dateFrom
start_date = []

#dateTo
finish_date = []

for date in years:
    start_date.append(int(str(date)+'01'))
    finish_date.append(int(str(date)+'12'))

print('검색시작일: ', start_date)
print('검색종료일: ', finish_date)

#institution
ins_name = '미래를 여는 청소년학회'
institution = quote_plus(ins_name)
print('검색학회: ', ins_name)

#displayCount
displayCount = 100
print('논문검색개수: ', displayCount)


################################# 크롤링 시작 #################################
print('#'*20, '크롤링 시작', '#'*20)

#논문별리스트(2차원)
information = []


tot_sum = 0
for i in tqdm(range(len(years))):
    time.sleep(0.1)

    #필요부분 리스트(1차원)
    year_list = []

    url = f'https://open.kci.go.kr/po/openapi/openApiSearch.kci?apiCode=articleSearch&key={key}&dateFrom={start_date[i]}&dateTo={finish_date[i]}&institution={institution}&displayCount={displayCount}'
    response = requests.get(url, verify=False)
    soup = bs(response.text, 'xml')

    #레코드 리스트
    records = soup.find_all('record')        
    
    for record in records:      
        need_list = []

        #저널네임        
        journal_name = record.find('journal-name').text
        need_list.append(journal_name)

        #출판연도
        pub_year = record.find('pub-year').text
        need_list.append(pub_year)

        #출판월
        pub_mon = record.find('pub-mon').text    
        need_list.append(pub_mon)

        #권
        volume = record.find('volume').text
        need_list.append(volume)

        #호
        if record.find('issue'):
            issue = record.find('issue').text
            need_list.append(issue)
        else:
            need_list.append('없음')

        #시작페이지
        start_page = record.find('fpage').text
        need_list.append(start_page)

        #끝페이지
        finish_page = record.find('lpage').text
        need_list.append(finish_page)

        #논문분류
        category = record.find('article-categories').text
        need_list.append(category)

        #국문제목
        kor_title = record.find('article-title', attrs={'lang': 'original'}).text.strip()
        need_list.append(kor_title)

        #영문제목
        if record.find('article-title', attrs={'lang': 'english'}):
            eng_title = record.find('article-title', attrs={'lang': 'english'}).text.strip()
            need_list.append(eng_title)
        else:
            need_list.append('없음')

        #저자
        author = record.find('author-group').text.strip().replace('\n', ', ')
        need_list.append(author)

        #국문초록
        if record.find('abstract', attrs={'lang': 'original'}):
            kor_abstract = record.find('abstract', attrs={'lang': 'original'}).text.strip()
            need_list.append(kor_abstract)    
        else:
            need_list.append('없음')

        #영문초록
        if record.find('abstract', attrs={'lang': 'english'}):
            eng_abstract = record.find('abstract', attrs={'lang': 'english'}).text.strip()
            need_list.append(eng_abstract)
        else:
            need_list.append('없음')

        #url
        link = record.find('url').text
        need_list.append(link)

        year_list.append(need_list)
    
    information.extend(year_list)
    # tot_sum += r

# print(tot_sum)

#################### API 사용 메뉴얼 ####################
검색시작일:  [201901, 202001, 202101]
검색종료일:  [201912, 202012, 202112]
검색학회:  미래를 여는 청소년학회
논문검색개수:  100
#################### 크롤링 시작 ####################


  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
 33%|███▎      | 1/3 [00:01<00:02,  1.06s/it]/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
 67%|██████▋   | 2/3 [00:01<00:00,  1.07it/s]/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarnin

In [ ]:
columns = ['저널이름', '출판연도', '출판월', '권', '호', '시작페이지', '끝페이지', '분류', '국문제목', '영문제목',
        '저자', '국문초록', '영문초록', '링크']

df = pd.DataFrame(data=information, columns=columns)
df

,저널이름,출판연도,출판월,권,호,시작페이지,끝페이지,분류,국문제목,영문제목,저자,국문초록,영문초록,링크
0,미래청소년학회지,2019,03,16,1,1,27,기타사회과학,강점관점 해결중심접근의 단기청소년쉼터 초기 사정도구 개발 - 청소년중심의 사례관리를...,The development of an initial assessment tool ...,"김윤나(서울사이버대학교), 이자영(한양사이버대학교)",본 연구는 단기청소년쉼터 기존 초기 사정도구의 한계점을 보완하고 좀 더 가출 청소년...,The purpose of this study is to develop an ini...,https://www.kci.go.kr/kciportal/ci/sereArticle...
1,미래청소년학회지,2019,03,16,1,29,56,기타사회과학,청소년기 학교폭력 피해 외상을 경험한 대학생의 외상 후 성장과 심리·사회적 적응의 ...,The Relationship between Posttraumatic Growth ...,"이응택(명지대학교), 이은경(명지대학교)",본 연구의 목적은 청소년기 학교폭력 피해 외상을 경험한 대학생들의 외상 후 성장과 ...,The goal of this study aims to examine the mod...,https://www.kci.go.kr/kciportal/ci/sereArticle...
2,미래청소년학회지,2019,03,16,1,57,86,기타사회과학,청소년 회복탄력성 검사개발 및 타당화,Development and Validation of the Resilience S...,"홍세희(고려대학교), 조기현(고려대학교), 이현정(고려대학교), 김효진(고려대학교)...",본 연구는 회복탄력성의 개념과 하위요인을 재정립하고 국내 청소년을 대상으로 하는 타...,The purposes of the present study were to refi...,https://www.kci.go.kr/kciportal/ci/sereArticle...
3,미래청소년학회지,2019,03,16,1,87,104,기타사회과학,"청소년의 불안, 회피애착과 학교폭력 방관·방어행동 간의 관계에서 이타성의 매개효과",A mediating effect of altruism on the relation...,"남궁미(서강대학교 교육대학원), 장재홍(서강대학교)","본 연구의 목적은 애착과 조력행동 간의 관계에 대한 기존연구를 토대로 하여, 청소년...",The purpose of the present study was to examin...,https://www.kci.go.kr/kciportal/ci/sereArticle...
4,미래청소년학회지,2019,06,16,2,1,21,기타사회과학,"메타인지 학습전략과 성취동기, 자기주도적 학습시간의 종단적 상호관계",Reciprocal relationships among metacognitive l...,"박정심(세종대학교), 박수원(세종대학교)",본 연구의 목적은 청소년기 자기주도적 학습에 중요한 구성요소인 메타인지 학습전략과 ...,The purpose of the current study was to examin...,https://www.kci.go.kr/kciportal/ci/sereArticle...
5,미래청소년학회지,2019,06,16,2,23,42,기타사회과학,지각된 인기도와 또래 괴롭힘 가해 및 방어 행동의 관계에서 내외통제성의 매개효과,The mediation effects of locus of control in t...,조유진(이화여자대학교),본 연구는 중학교에 재학 중인 청소년들을 대상으로 이들의 지각된 인기도와 또래괴롭힘...,This study analyzed the mediating effects of l...,https://www.kci.go.kr/kciportal/ci/sereArticle...
6,미래청소년학회지,2019,06,16,2,43,63,기타사회과학,청소년지도사의 윤리의식수준에 대한 실태조사 연구,A Study on the level of ethical awareness of y...,권일남(명지대학교),본 연구는 청소년지도사의 윤리의식과 기관의 대응방안을 알아보는 데 있으며 주요 내용...,The purpose of this study was to examine the e...,https://www.kci.go.kr/kciportal/ci/sereArticle...
7,미래청소년학회지,2019,06,16,2,65,81,기타사회과학,대학생의 진로탄력성 척도 개발 연구,The development of college students’ career re...,"조아미(명지대학교), 진영선(명지대학교), 임정아(명지대학교)","본 연구의 목적은 진로탄력성 개념과 구성요인을 정의하고, 이를 측정하는 대학생용 진...",The purpose of this study was to define the co...,https://www.kci.go.kr/kciportal/ci/sereArticle...
8,미래청소년학회지,2019,09,16,3,1,21,기타사회과학,자녀의 연령에 따른 부모와의 애착이 대인관계에 미치는 영향: 개념적 모형의 탐색,Exploration of conceptual model to child-paren...,"김선미(충북대학교 아동복지학과), 김영희(충북대학교)",본 연구는 부모와의 애착이 자녀의 대인관계에 미치는 상대적 영향력을 개념적 모형(단...,The purpose of this study was to examine the e...,https://www.kci.go.kr/kciportal/ci/sereArticle...
9,미래청소년학회지,2019,09,16,3,23,49,기타사회과학,청소년의 온·오프라인 학교폭력 가해경험과 피해경험 관련 변인 간의 횡·종단적 구조분석,A structural Analysis the cross-sectional and ...,"성정혜(경북대학교), 김춘경(경북대학교)",본 연구는 청소년의 온·오프라인 학교폭력의 원인변인을 규명하고자 오프라인 학교폭력의...,The purpose of this study was to analyze the s...,https://www.kci.go.kr/kciportal/ci/sereArticle...


In [ ]:
df.to_excel('/content/게재된 논문.xlsx')